In [1]:
%pylab inline
import numpy as np
import scipy as sp
import tensorflow as tf
import pandas as pd
import seaborn as sns
import itertools,time
import sys, os
from collections import OrderedDict
from copy import deepcopy
import numpy as np
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt
import itertools
import pickle
import tensorflow.contrib.distributions.python.ops as ops
from tensorflow.contrib.distributions.python.ops import operator_pd_full
import time

Populating the interactive namespace from numpy and matplotlib


/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
dataset_tr = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/NTP/process_UCI/M_nips.full_docs.mat'
data_tr = sp.io.loadmat(dataset_tr)['M'].T.toarray()

vocab_ = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/NTP/process_UCI/vocab.nips.txt'
vocab_ = open(vocab_,'r').readlines()
vocab=np.array([word for word in vocab_])

print data_tr.shape, vocab.shape

(1500, 12419) (12419,)


In [3]:
vocab_size=len(vocab)

In [4]:
n_samples_tr = data_tr.shape[0]
docs_tr = data_tr

In [5]:
def xavier_init(fan_in, fan_out, constant=1): 
    """ Xavier initialization of network weights"""
    # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
    low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                             minval=low, maxval=high, 
                             dtype=tf.float32)
def log_dir_init(fan_in, fan_out,topics=50): 
    return tf.log((1.0/topics)*tf.ones([fan_in, fan_out]))

tf.reset_default_graph()
class VariationalAutoencoder(object):
    """ 
    See "Auto-Encoding Variational Bayes" by Kingma and Welling for more details.
    """
    def __init__(self, network_architecture, transfer_fct=tf.nn.softplus, 
                 learning_rate=0.001, batch_size=100):
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        print 'Learning Rate:', self.learning_rate
        
        # tf Graph input
        self.x = tf.placeholder(tf.float32, [None, network_architecture["n_input"]])
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.h_dim = float(network_architecture["n_z"])
        self.nz=network_architecture["n_z"]
        
        self.a = np.ones((self.batch_size , self.h_dim)).astype(np.float32)/self.h_dim
        
#         self.mu2 = tf.constant(0.0)
#         self.var2 = tf.constant(  np.ones(self.batch_size,self.h_dim).astype(np.float32)  )
        
        
        
        # Create autoencoder network
        self._create_network()
        # Define loss function based variational upper-bound and 
        # corresponding optimizer
        self._create_loss_optimizer()
        
        # Initializing the tensor flow variables
        init = tf.initialize_all_variables()

        # Launch the session
        self.sess = tf.InteractiveSession()
        self.sess.run(init)
    
    def _create_network(self):
        # Initialize autoencode network weights and biases
        self.network_weights = self._initialize_weights(**self.network_architecture)
        '''get mean and sigma'''
            
        self.z_mean = \
            self.mean_network(self.network_weights["sub_01"], 
                                      self.network_weights["biases_sub_01"])
        
        self.log_chol = \
            self.sigma_network(self.network_weights["sub_02"], 
                                      self.network_weights["biases_sub_02"])

        # Draw one sample z from Gaussian distribution

        eps = tf.random_normal((1, self.nz), 0, 1, dtype=tf.float32)
        '''Adding RT for subtopics'''
        
        self.chol = tf.exp(self.log_chol)
        
        self.sigma = tf.batch_matmul(self.chol,self.chol,adj_y=True)
        
#         self.sigma_log_det = 2*tf.reduce_sum(tf.log(tf.diag(tf.cholesky(self.sigma))),1)
        
        temp_calc=tf.squeeze(tf.map_fn(lambda i:tf.matmul(eps,tf.gather(tf.sqrt(self.sigma) ,i) ),
                                                   tf.constant(np.arange(self.batch_size)), dtype=tf.float32))
        
        self.z = tf.add(self.z_mean, temp_calc)

        
        '''Adding reconstruction'''
        
        self.x_reconstr_mean = \
            self._generator_network(self.z,self.network_weights["weights_gener"],
                                    self.network_weights["biases_gener"])
            
        
            
    def _initialize_weights(self, n_hidden_recog_1, n_hidden_recog_2, 
                            n_hidden_gener_1,  
                            n_input, n_z):
        all_weights = dict()

        with tf.variable_scope("sub01"):
            all_weights['sub_01'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_mean': tf.get_variable('out_mean',[n_hidden_recog_2, n_z])}
            all_weights['biases_sub_01'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([n_z], dtype=tf.float32))
            }
        with tf.variable_scope("sub02"):
            all_weights['sub_02'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[n_hidden_recog_2, n_z*n_z])
            }
            all_weights['biases_sub_02'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([n_z*n_z], dtype=tf.float32))
            }
        with tf.variable_scope("gen"):
            all_weights['weights_gener'] = {
                'h2': tf.Variable(xavier_init(n_z, n_hidden_gener_1))
            }
            all_weights['biases_gener'] = {
                'b2': tf.Variable(tf.zeros([n_hidden_gener_1], dtype=tf.float32))
            }
        return all_weights
            
    def mean_network(self, weights, biases):
        # Generate probabilistic encoder (recognition network)
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.x, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        layer_do = tf.nn.dropout(layer_2, self.keep_prob)
        
        z_mean = tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_mean']),
                        biases['out_mean']))    
        
        return z_mean
    
    def sigma_network(self, weights, biases):
        # Generate probabilistic encoder (recognition network)
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.x, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        layer_do = tf.nn.dropout(layer_2, self.keep_prob)
        
#         z_sigma_sq = \
#             tf.exp(tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_log_sigma']), 
#                    biases['out_log_sigma'])))  
        z_log_sigma_sq = \
            tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_log_sigma']), 
                   biases['out_log_sigma'])) 
        
        return tf.reshape(z_log_sigma_sq, [self.batch_size,self.nz, self.nz])

    def _generator_network(self,z, weights, biases):
        # Generate probabilistic decoder (decoder network)
        self.layer_do_0 = tf.nn.dropout(tf.nn.softmax(tf.contrib.layers.batch_norm(z)), self.keep_prob)
#         x_reconstr_mean = tf.add(tf.matmul(self.layer_do_0, 
#                                            tf.nn.softmax(tf.contrib.layers.batch_norm(weights['h2']))),0.0)
        x_reconstr_mean = tf.nn.softmax(tf.contrib.layers.batch_norm(tf.add(
                    tf.matmul(self.layer_do_0, weights['h2']),0.0)))
    
        return x_reconstr_mean

            
    def _create_loss_optimizer(self):
        
        self.x_reconstr_mean+=1e-10
     
        reconstr_loss = \
            -tf.reduce_sum(self.x * tf.log(self.x_reconstr_mean),1)#/tf.reduce_sum(self.x,1) 
            
#         temp_s=self.sigma.add_to_tensor(0.0)

        temp_trace=tf.squeeze(tf.map_fn(lambda i:tf.trace(tf.gather(self.sigma,i)),
                                                   tf.constant(np.arange(self.batch_size)), dtype=tf.float32))
        
        temp_log_det=tf.squeeze(tf.map_fn(lambda i:2*tf.reduce_sum(tf.log(tf.diag_part(tf.gather(self.chol,i)))),
                                                   tf.constant(np.arange(self.batch_size)), dtype=tf.float32))
        print temp_trace
        print temp_log_det
        
#         latent_loss = 0.5*( temp_trace+\
#         tf.reduce_sum( tf.mul(tf.div((self.mu2 - self.z_mean),self.var2),
#                   (self.mu2 - self.z_mean)),1) - self.h_dim +\
#                            tf.reduce_sum(tf.log(self.var2),1)  - tf.log(temp_det+1e-10))

        
        latent_loss = 0.5*( temp_trace +\
        tf.reduce_sum(tf.square(self.z_mean),1) - self.h_dim +\
                           tf.log(1.0) - temp_log_det )


        self.cost = tf.reduce_mean(reconstr_loss) \
    + tf.reduce_mean(latent_loss) # average over batch
        
        # Use ADAM optimizer
        self.optimizer = \
            tf.train.AdamOptimizer(learning_rate=self.learning_rate,beta1=0.9).minimize(self.cost)
        
    def partial_fit(self, X):
        """Train model based on mini-batch of input data.
        
        Return cost of mini-batch.
        """
#         opt, cost,emb,kld = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2'],
#                                           self.kld),feed_dict={self.x: X,self.keep_prob: .9})
        opt,cost,emb = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2']
                                      ),feed_dict={self.x: X,self.keep_prob: .7})
        return cost,emb
    
    def test(self, X):
        """Test the model and return the lowerbound on the log-likelihood.
        """
        cost,e1,e2 = self.sess.run((self.cost,tf.nn.softmax(self.z1),tf.nn.softmax(self.z2)),
                                        feed_dict={self.x: np.expand_dims(X, axis=0),self.keep_prob: 1.0})
        return cost,e1,e2


In [6]:
tf.reset_default_graph()
cost_plot=[]
def train(network_architecture, learning_rate=0.0001,
          batch_size=100, training_epochs=10, display_step=5):
    vae = VariationalAutoencoder(network_architecture, 
                                 learning_rate=learning_rate, 
                                 batch_size=batch_size)
    emb=0
    avg_kld=0
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(n_samples_tr / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = minibatches.next()
            # Fit training using batch data
            cost,emb = vae.partial_fit(batch_xs)
#             print cost
#             sys.exit()
            # Compute average loss
            avg_cost += cost / n_samples_tr * batch_size
            
            if np.isnan(avg_cost):
                print epoch,i,np.sum(batch_xs,1).astype(np.int),batch_xs.shape
                return vae,emb
                sys.exit()

        # Display logs per epoch step
        if epoch % display_step == 0:
            cost_plot.append(avg_cost)
            print "Epoch:", '%04d' % (epoch+1), \
                  "cost=", "{:.9f}".format(avg_cost)

    return vae,emb

In [7]:
tf.reset_default_graph()
network_architecture = \
    dict(n_hidden_recog_1=100, # 1st layer encoder neurons
         n_hidden_recog_2=100, # 2nd layer encoder neurons
         n_hidden_gener_1=docs_tr.shape[1], # 1st layer decoder neurons
         n_input=docs_tr.shape[1], # MNIST data input (img shape: 28*28)
         n_z=50)  # dimensionality of latent space

batch_size=200
learning_rate=0.003
def create_minibatch(data):
    rng = np.random.RandomState(10)
    
    while True:
        # Return random data samples of a size 'minibatch_size' at each iteration
        ixs = rng.randint(data.shape[0], size=batch_size)
        yield data[ixs]
        
minibatches = create_minibatch(docs_tr.astype('float32'))
start=time.time()
vae,emb = train(network_architecture, training_epochs=500,batch_size=batch_size,learning_rate=learning_rate)
# plt.plot(cost_plot)
print 'it took',str(time.time()-start),'seconds'

'''
Change log: decoder has no dropout or additional tranformations
'''

Learning Rate: 0.003
Tensor("Squeeze_1:0", shape=(200,), dtype=float32)

/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)
/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



Tensor("Squeeze_2:0", shape=(200,), dtype=float32)
Epoch: 0001 cost= 20874.189062500
Epoch: 0006 cost= 15584.535546875
Epoch: 0011 cost= 14441.281770833
Epoch: 0016 cost= 13812.410937500
Epoch: 0021 cost= 13282.498958333
Epoch: 0026 cost= 12630.240104167
Epoch: 0031 cost= 12210.594921875
Epoch: 0036 cost= 11617.027994792
Epoch: 0041 cost= 10928.061328125
Epoch: 0046 cost= 10834.406901042
Epoch: 0051 cost= 10534.898958333
Epoch: 0056 cost= 10744.641015625
Epoch: 0061 cost= 10501.440755208
Epoch: 0066 cost= 10338.390755208
Epoch: 0071 cost= 10197.922656250
Epoch: 0076 cost= 10244.702864583
Epoch: 0081 cost= 10059.734895833
Epoch: 0086 cost= 10065.782421875
Epoch: 0091 cost= 9908.077994792
Epoch: 0096 cost= 9874.081119792
Epoch: 0101 cost= 9752.851822917
Epoch: 0106 cost= 9823.053125000
Epoch: 0111 cost= 9652.087890625
Epoch: 0116 cost= 9717.519140625
Epoch: 0121 cost= 9590.664973958
Epoch: 0126 cost= 9574.706640625
Epoch: 0131 cost= 9532.918229167
Epoch: 0136 cost= 9427.592578125
Epoch:

'\nChange log: decoder has no dropout or additional tranformations\n'

In [11]:
def print_top_words(beta, feature_names, n_top_words=5):
    for i in range(len(beta)):
        print(" ".join([feature_names[j]
            for j in beta[i].argsort()[:-n_top_words - 1:-1]]))

print_top_words(emb, "".join(vocab).split('\n'))

chip cmos capacitor synapse mos
svm wahba hyperparameter pima hyperprior
voltages resistor circuit mead westervelt
margin svm tangent smola adaboost
welch pomdp suffix frasconi towell
oscillation olfactory hopf oscillator excitatory
policy mdp pomdp parr kaelbling
market portfolio bagging sharpe hint
maass spiking epsp theorem firing
photoreceptor bipolar retina silicon delbriick
cleereman eeckman grammatical string genetic
backgammon rollout gammon bertsekas player
hyperparameter obd barber retraining surgeon
hmm bourlard pdf phonetic vocabulary
variational factorized saul belief boltzmann
christof timothy manfred andrew yoram
ica cardoso blind kurtosis cichocki
rangarajan pott relational pami hancock
saad replica student biehl realizable
adversary mistake tracker warmuth littlestone
dominance ocular stryker monocular cortical
membrane conductance potassium ganglion depolarization
barto obstacles bradtke reinforcement watkin
musical melody chorale amino protein
retrieval document dict

In [17]:

cost=[]
e1=[]
e2=[]
idx=0
for doc in docs_te:
    idx+=1
    doc = doc.astype('float32')
    n_d = np.sum(doc)
    c,a,b=vae.test(doc)
    cost.append(c/n_d)
    e1.append(a)
    e2.append(b)
    if idx%1000==0:
        print idx
    
# print (np.exp(np.mean(np.array(cost))))
# print np.mean(e1,0)
# print np.mean(e2,0)

1000
2000
3000
4000
5000
6000
7000


In [52]:
with open('nips.txt','w') as o:

    for doc in docs_tr:
        doc_o=[]
        j=-1
        for wc in doc:
            j+=1
            for i in range(int(wc)):
                doc_o.append(vocab[j].strip())
        o.write(" ".join(doc_o)+'\n')

    

In [19]:
pickle.dump(cost,open('cost','w'))
pickle.dump(e1,open('e1','w'))
pickle.dump(e2,open('e2','w'))